# 📘 OpenAI Tools + AKShare 实现：查询“小米股价”示例

本 notebook 示例演示如何基于 **OpenAI Function Calling** + **AKShare** 实现：

👉 **用户问：小米股价多少？（或任意股票代码）**

模型自动调用工具 → 工具使用 AKShare 查询实时行情 → 返回结果。


In [ ]:
import json
from typing import Optional

### -----------------------------
### 1. OpenAI Tools Schema
### -----------------------------

tool_schema = {
    "name": "get_stock_price",
    "description": "查询实时股票价格，支持A股与港股（如小米：1810.HK）",
    "parameters": {
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "股票代码，如 1810.HK（小米）、600519（贵州茅台）"
            }
        },
        "required": ["symbol"]
    }
}

print("工具 Schema：")
print(json.dumps(tool_schema, ensure_ascii=False, indent=2))

In [1]:
!pip install akshare

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 2.3 MB/s eta 0:00:00a 0:00:01
  Created wheel for jsonpath: filename=jsonpath-0.82.2-py3-none-any.whl size=5615 sha256=c0e1dac8faa5c41429c54c5d83d24827d024e894222813e5f82e358debc76dc8
  Stored in directory: /private/var/folders/lq/fr2l0cz521q9jx7kv7wfswvh0000gn/T/pip-ephem-wheel-cache-zje6zzjc/wheels/77/0d/9c/0254ddd8f4fe128f747e9647084c9a6ef6da3322020dfb4979
Successfully built jsonpath
  Attempting uninstall: aiosignal
    Found existing installation: aiosignal 1.2.0
    Uninstalling aiosignal-1.2.0:
      Successfully uninstalled aiosignal-1.2.0
  Attempting uninstall: aiohappyeyeballs
    Found existing installation: aiohappyeyeballs 2.4.0
    Uninstalling aiohappyeyeballs-2.4.0:
      Successfully uninstalled aiohappyeyeballs-2.4.0
  Attem

In [ ]:
### -----------------------------
### 2. 工具实现（AKShare）
### -----------------------------

def get_stock_price(symbol: str):
    """
    查询股票当前价格
    小米股票代码为 1810.HK
    """
    try:
        import akshare as ak

        # 判断是否为港股（如 1810.HK）
        if symbol.endswith('.HK'):
            df = ak.stock_hk_quote()
            row = df[df['代码'] == symbol.replace('.HK', '')]
        else:
            df = ak.stock_zh_a_spot()
            row = df[df['代码'] == symbol]

        if row.empty:
            return {"error": f"未找到股票：{symbol}"}

        return row.to_dict(orient='records')[0]

    except Exception as e:
        return {"error": str(e)}


### 示例：查询小米股价（1810.HK）
print(get_stock_price("1810.HK"))

在你的实际 OpenAI Tools 代码中，只需要把 `get_stock_price` 注册为工具即可。